# Managing your FABRIC Credentials 

#### [1. About](#about)
#### [2. Step-by-step Setup](#setup)
#### [3. Next Steps](#next)

<a id='about'></a>
# About

The FABRIC API is used via proxy objects that manage connections to the control framework and allow the user to reserve and access physical resources. For any experimentation on the FABRIC testbed it is therefore required take a few steps to establish a connection gateway to the system. This is done via:

- A credential manager proxy, `Slice Manager`. 
- Authentication `Tokens`.
- `SSH Keys`. 

In [ ]:
help(SliceManager)

### Tokens    
Fabric uses two kinds of tokens:

- `Refresh Token`: When a user logins to Jupyterhub after authenticating against CILogon, an initial refresh token is automatically derived. This token is available as the environment variable `CILOGON_REFRESH_TOKEN` and is used to generate the initial Identity Token. On every call to the [refresh_tokens()](#refresh) method, Fabric Refresh Token is updated. `Each Refresh Token is valid for 24 hours.`

- `Identity Token`: The control/measurement framework APIs require an Identity Token. Identity tokens can be generated from a valid Refresh Token at any time. On creation/refresh The new ID token is added to the users environment variables. `Each Identity Token is valid upto an hour.`

**Note:** Tokens are not used as explicit arguments to FABRIC API functions but need to be generated/refreshed and saved manually so they can do their job in the background. Without valid tokens in the environment a user cannot make slice reservation!

### SSH 
To access the physical nodes at FABRIC, the user needs an `SSH Key`.`SSH` is used when requesting to create a slice and when configuring nodes.

<a id='setup'></a>
# Step-by-step Setup

### 1. Save the Initial Refresh Token

The first step is to retrieve or set the `Refresh Token` (exprires 24 hours after login).

In [ ]:
import os

fabric_refresh_token=None
%store -r fabric_refresh_token  #store OR retrieve -r

if fabric_refresh_token is None:
    fabric_refresh_token=os.environ['CILOGON_REFRESH_TOKEN']
    %store fabric_refresh_token

print("Refresh Token: {}".format(fabric_refresh_token))

### 2. Create a Slice Manager

Using the `Refresh Token`, an instance of `Slice Manager` is created.

Environment variables `FABRIC_CREDMGR_HOST` and `FABRIC_ORCHESTRATOR_HOST` are passed to the constructor to set the values for the Credential Manager Host and Orchestrator Client Host in `Slice Manager`.

Users can request tokens with different Project and Scopes by altering `project_name` and `scope` parameters.

In [ ]:
import os
import json
from fabrictestbed.slice_manager import SliceManager, Status

credmgr_host = os.environ['FABRIC_CREDMGR_HOST']
orchestrator_host = os.environ['FABRIC_ORCHESTRATOR_HOST']
print(f"CM Host: {credmgr_host} Orchestrator Host: {orchestrator_host}")

# Create Slice Manager
slice_manager = SliceManager(oc_host=orchestrator_host, cm_host=credmgr_host, 
                             refresh_token=fabric_refresh_token, project_name='all', scope='all')

<a id='refresh'></a>
### 3. Generate/Refresh ID Token

After `Slice Manager` has been created, the `ID Token` can be generated or refreshed. **Since ID Tokens expire after one hour, the user should rememeber to re-run this command frequently.** 

**Note:** the same command, `slice_manager.refresh_tokens()`, is used both to generate the initial ID Token and to refresh the ID and Refresh tokens.

In [1]:
try:
    id_token, refresh_token = slice_manager.refresh_tokens()
except Exception as e:
    print("Exception occurred while getting tokens:{}".format(e))

fabric_refresh_token=slice_manager.get_refresh_token()
print()
print("New Refresh Token: {}".format(fabric_refresh_token))
print()
print("New Id Token: {}".format(fabric_id_token))
print()
print("Stored new Refresh and ID Tokens")
%store fabric_refresh_token 
%store fabric_id_token
print()

Exception occurred while getting tokens:name 'slice_manager' is not defined


NameError: name 'slice_manager' is not defined

### 4. Find and Save the SSH Key

An `SSH Key` is needed to access the users physical nodes at FABRIC. A file with the key is already present in the environment under `/home/fabric/.ssh/id_rsa.pub` but needs to be read into a variable. The SSH Key should be passed as an argument when requesting to make a new slice.

In [1]:
ssh_key = None
with open ("/home/fabric/.ssh/id_rsa.pub", "r") as myfile:
    ssh_key=myfile.read().strip()

#USAGE:
    #In requesting slice from Orchestrator
    #status, reservations = slice_manager.create(slice_name=slice_name, slice_graph=slice_graph, ssh_key=ssh_key)

### 5. SSH and the Paramiko library

For the Configuring of nodes, a more elaborate SSH framework is needed. Using the python `paramiko library` the user creates an `SSH Client` object and retrieves the key from the .shh folder. Through this client a connection is made to the Management IP of each sliver (each node) allowing the client to execute a desired script on the slivers/nodes.

In [3]:
import paramiko 

key = paramiko.RSAKey.from_private_key_file("/home/fabric/.ssh/id_rsa")
client = paramiko.SSHClient()
client.load_system_host_keys()
client.set_missing_host_key_policy(paramiko.MissingHostKeyPolicy())

#USAGE:
    #when running scripts on nodes
    #for sliver in slivers:
    #    node_name = sliver.name
    #    management_ip = sliver.management_ip

    #    print("Node {0} IP {1}".format(node_name, management_ip))

    #    client.connect(management_ip,username='ubuntu',pkey = key)

    #    stdin, stdout, stderr = client.exec_command('echo \"' + script + '\" > script.sh; chmod +x script.sh; sudo ./script.sh')
    #    print ('')
    #    print (str(stdout.read(),'utf-8').replace('\\n','\n'))

    #    client.close()

<a id='next'></a>
# Next Steps

Continue getting to know FABRIC!

- [Get Available resources](get_available_resources.ipynb)<br>
- [Single Node](single_node.ipynb)<br>
- [Renew Existing Slice](renew_existing_slice.ipynb)<br>
- [Delete Existing Slice](delete_existing_slice.ipynb)<br>